In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import librosa
from sklearn.metrics import accuracy_score
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# load the preloaded process-audio
import pickle
with open('/kaggle/input/process-audio/train_processed_audio.pkl','rb') as f:
    data = pickle.load(f)

In [3]:
with open('/kaggle/input/process-audio/test_processed_audio.pkl','rb') as file:
    test_data = pickle.load(file)

test_data = pd.DataFrame([test_data]).T
test_data.reset_index(inplace=True)
test_data.rename(columns={'index':'labels',0:'embeddings'},inplace=True)
test_data.head(5)

,labels,embeddings
0,41663,"[[0.0, 1.2797491e-07, 0.00021587347, 0.0025362..."
1,yecspi2,"[[4.974095, 58.645084, 198.50327, 369.67136, 3..."
2,baymac,"[[1.6719003e-09, 0.009455838, 1.3032254, 8.573..."
3,gybmar,"[[2.0096888e-05, 0.03433669, 0.18154305, 0.108..."
4,trokin,"[[0.0, 2.278743e-15, 3.5270189e-06, 0.01068996..."


In [4]:
processed_data = pd.DataFrame([data]).T
processed_data.reset_index(inplace=True)
processed_data.rename(columns={'index':'labels',0:'embeddings'},inplace=True)
processed_data.shape

(206, 2)

In [5]:
final_data = pd.concat([processed_data,test_data],axis=0)
final_data.shape

(396, 2)

In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
encoded_target = le.fit_transform(final_data.labels)

In [7]:
encoded_target

array([ 63, 197,  97,  80, 121, 113, 129, 168, 199, 134, 143, 179, 151,
       148, 124, 194, 123, 181, 201, 176, 185, 193, 171, 155,  13, 190,
        84,  90,  77, 147, 183, 149, 178, 198, 111, 180, 202, 173,  99,
       144,  82,  60, 139,  70, 153, 102, 188,  67,  72,  71, 203, 166,
       163, 136, 152, 200, 164,  66,  89, 204, 119, 169, 131, 150, 104,
        36, 101, 138,  30,  68,  61, 145,  87, 187, 174, 177,  79, 132,
        92,  14, 162, 126, 161, 195,  93, 140, 158, 172,  88, 205, 122,
       196,  69, 100,  19,  98,  78, 175,  86,  91, 109,  96, 142,  32,
        73, 157,  38, 165, 184,  76, 106,  64, 110, 170, 191, 118, 107,
        23,  55, 167,   4, 125,  75, 133, 120, 115,  58, 192, 160, 114,
       116,  56, 146,  85,  27,  62,  37,  12, 105,  31,  45, 112,  74,
        41,  22,  16,  94,  65,  20, 127, 189,  95, 159,   9, 186, 128,
       103,   5,  44,  81, 130,  29,  18, 154, 108, 141,  35,  28,  83,
        52, 117, 156, 135,   6,  48,  43,  15,  47,  42,  24,  3

In [8]:
# import tensorflow as tf

# # Set the seed value for experiment reproducibility.
# seed = 42
# tf.random.set_seed(seed)
# np.random.seed(seed)

# embeddings = np.stack(final_data['embeddings'].values)
# targets = encoded_target


# x = tf.convert_to_tensor(embeddings)
# y = tf.convert_to_tensor(targets)

# if len(y.shape) == 2:
#     y = tf.squeeze(y, axis=1)

# tf_data = tf.data.Dataset.from_tensor_slices((x, y)).batch(32)

In [9]:
# for x,y in tf_data.take(1):
#     print(x.shape)
#     print(y.shape)

In [10]:
# convert dataset into train and val set

# tf_data = tf_data.shuffle(buffer_size = tf_data.cardinality().numpy())
# train_df = tf_data.take(10)
# val_df = tf_data.skip(3)

In [11]:
# for x, y in train_df.take(1):
#     print("x shape:", x[0].shape)
#     print("y shape:", y[0].shape)

In [12]:
# training arrays
X = np.stack(final_data['embeddings'].values)
y = encoded_target

# split data into train and test split
split_size = int(0.7*X.shape[0])
X_train,X_val = X[:split_size],X[split_size:]
y_train,y_val = y[:split_size],y[split_size:]

In [13]:
X_train.shape

(277, 128, 313)

In [14]:
# model training
from tensorflow.keras import layers,models

model = models.Sequential(
    [
        layers.Input(shape=(128,313)),
        layers.Conv1D(32,kernel_size=3,activation='relu'),
        layers.MaxPooling1D(2),

        layers.Conv1D(64,kernel_size=3,activation='relu'),
        layers.MaxPooling1D(2),

        layers.Flatten(),
        layers.Dense(128,activation='relu'),
        layers.Dense(206,activation='softmax')
        
    ]
    
)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
callbacks = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=5)
model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=500,callbacks=[callbacks])

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 579ms/step - accuracy: 0.0000e+00 - loss: 7.4458 - val_accuracy: 0.0084 - val_loss: 6.3974
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1711 - loss: 4.5904 - val_accuracy: 0.0084 - val_loss: 6.2503
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3705 - loss: 3.9117 - val_accuracy: 0.0336 - val_loss: 6.8383
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3907 - loss: 3.7242 - val_accuracy: 0.0420 - val_loss: 7.9201
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4616 - loss: 3.1775 - val_accuracy: 0.0336 - val_loss: 9.4557
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4883 - loss: 2.9614 - val_accuracy: 0.0336 - val_loss: 11.4803
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4689 - loss: 2.9856 - val_accuracy: 0.0336 - val_loss: 13.8859
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5873 - loss: 2.5223 - val_accuracy: 0.0336 - val_loss:

In [15]:
# processing using librosa
def load_audio(file):
    audio,sr = librosa.load(file,sr=32000)
    return audio

# amplitude envelope
def amplitude_envelope(signal,frame_size,hop_length):
    envelope = list()
    # calculate amplitude frame
    for i in range(0,len(signal),hop_length):
        amplitude_env_frame = max(signal[i:i+frame_size])
        envelope.append(amplitude_env_frame)
    return envelope


# audio to mel spectrogram
def audio2melspec(audio_data):
    """Returns the mel spec for audio file"""
    if np.isnan(audio_data).any():
        mean_signal = np.nanmean(audio_data)
        audio_data = np.nan_to_num(audio_data, nan=mean_signal)

    mel_spec = librosa.feature.melspectrogram(
        y=audio_data,
        sr=32000,
        n_fft=1024,
        hop_length=500,
        n_mels=128,
        fmin=40,
        fmax=15000,
        power=2.0
    )
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_spec_db


# complete processsing
def preprocess(audio_file):
    """Returns the processed audio files"""
    loaded_aud = load_audio(audio_file)
    target_audio = int(5.0*32000)
    if len(loaded_aud) < target_audio:
        n_copy = np.math.ceil(target_audio/len(loaded_aud))
        if n_copy > 1:
            loaded_aud = np.concatenate([loaded_aud] * n_copy)

    start_idx = 0 
    end_idx=start_idx + target_audio
    center_audio = loaded_aud[start_idx:end_idx]

    if len(center_audio) < target_audio:
        center_audio = np.pad(center_audio, 
                             (0, target_samples - len(center_audio)), 
                             mode='constant')
        
    mel_aud = audio2melspec(center_audio)
    return mel_aud

# Load label encoder used during training
primary_labelencoder = LabelEncoder()
class_labels = sorted(os.listdir('/kaggle/input/birdclef-2025/train_audio/'))
primary_labelencoder.fit(class_labels)  # Fit encoder on original labels

# Paths
test_dir = '/kaggle/input/birdclef-2025/test_soundscapes'
sample_sub = pd.read_csv('/kaggle/input/birdclef-2025/sample_submission.csv')

# Final prediction storage
submission_rows = []

# Iterate through soundscapes
for filename in sorted(os.listdir(test_dir)):
    print(filename)
    if not filename.endswith('.ogg'):
        continue
    try :
        file_path = os.path.join(test_dir, filename)
        y, _ = librosa.load(file_path, sr=sr)
        soundscape_id = filename.replace('.ogg', '')
    
        # Extract features and reshape
        mfcc = preprocess(y)
        input_features = np.expand_dims(mfcc, axis=0)
        print(input_features)

        # Predict probabilities
        probs = model.predict(input_features, verbose=0)[0]

        # Convert indices back to original species IDs
        mapped_labels = primary_labelencoder.inverse_transform(range(len(probs)))

        # Create a dictionary of {species_id: probability}
        prob_dict = dict(zip(mapped_labels, probs))

        # Ensure submission columns match sample_submission.csv order
        ordered_probs = [prob_dict.get(species, 0) for species in sample_sub.columns[1:]]

        # Append row
        row_id = f"{soundscape_id}_{i + duration}"
        submission_rows.append([row_id] + ordered_probs)
    except Exception as e:
        print(e)

# Build submission DataFrame
submission_df = pd.DataFrame(submission_rows, columns=['row_id'] + list(sample_sub.columns[1:]))

# Save CSV
submission_df.to_csv("submission.csv", index=False)
print("submission.csv created")
submission_df.head(5)

readme.txt
submission.csv created


,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
